In [151]:
import cv2 as cv
import numpy as np
from ultralytics import YOLO

model = YOLO(
    "/Users/amadyarov/Documents/Coding/projects/board-destroyer/model/weights/best.pt"
)

In [152]:
def draw_boxes(board, xyxy_coords, class_names):
    for box, label in zip(xyxy_coords, class_names):
        x1, y1, x2, y2 = box
        cv.rectangle(board, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv.putText(
            board, label, (x1, y1 + 20), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 6
        )

    cv.imshow("Result", board)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [153]:
piece_names = {
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "P",
    "black-king": "k",
    "black-queen": "q",
    "black-rook": "r",
    "black-bishop": "b",
    "black-knight": "n",
    "black-pawn": "p",
}

In [154]:
def find_pieces(board):
    pos = np.zeros((8, 8), dtype=np.dtype("U1"))
    res = model.predict(board)[0]
    coords = res.boxes.xyxy.numpy().astype(int)[:, 0:2]
    labels = [model.names[int(c)] for c in res.boxes.cls]
    board_height, board_width, _ = board.shape

    for i in range(len(coords)):
        x, y = coords[i]
        posX = round(8 * x / board_width)
        posY = round(8 * y / board_height)
        pos[posY, posX] = piece_names[labels[i]]

    return pos

In [158]:
board = cv.imread("assets/cb3.png")
board = cv.resize(board, (416, 416))
finds = find_pieces(board)
print(finds)


0: 640x640 2 black-bishops, 1 black-king, 2 black-knights, 8 black-pawns, 1 black-queen, 2 black-rooks, 2 white-bishops, 1 white-king, 2 white-knights, 8 white-pawns, 1 white-queen, 2 white-rooks, 161.6ms
Speed: 1.9ms preprocess, 161.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


['black-bishop', 'black-pawn', 'white-pawn', 'white-bishop', 'black-rook', 'black-king', 'black-knight', 'white-bishop', 'white-knight', 'white-pawn', 'white-knight', 'black-bishop', 'black-rook', 'black-pawn', 'white-queen', 'white-rook', 'black-pawn', 'white-rook', 'black-knight', 'black-queen', 'white-pawn', 'black-pawn', 'black-pawn', 'white-pawn', 'white-pawn', 'black-pawn', 'white-pawn', 'white-pawn', 'black-pawn', 'black-pawn', 'white-pawn', 'white-king']
[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']
 ['P' 'P' 'P' 'P' 'P' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]
